In [1]:
#!pip install bitsandbytes

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import warnings
warnings.filterwarnings('ignore')

In [3]:
!nvidia-smi


Tue Aug 12 08:32:36 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   52C    P8             17W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
print(torch.cuda.get_device_name(0))
print(torch.cuda.get_device_properties(0))

NVIDIA L4
_CudaDeviceProperties(name='NVIDIA L4', major=8, minor=9, total_memory=22692MB, multi_processor_count=58, uuid=6647136b-b8c3-82a6-9be4-77f2c82d7672, L2_cache_size=48MB)


In [5]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [6]:
# Model configuration
model_name = "microsoft/Phi-4"

In [7]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

In [8]:
# Load model in 4-bit to save VRAM
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # Use half precision to save memory
    device_map="auto",          # Automatically distribute across available GPUs
    trust_remote_code=True,
    load_in_4bit=True,         # Use 4-bit quantization to reduce memory usage
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

model-00005-of-00006.safetensors:   0%|          | 0.00/4.77G [00:00<?, ?B/s]

model-00006-of-00006.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

...

model-00003-of-00006.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

In [9]:
def generate_response(prompt, max_length=512, temperature=0.7, top_p=0.9):
    """
    Generate a response using the Phi-4 model

    Args:
        prompt (str): Input text prompt
        max_length (int): Maximum length of generated text
        temperature (float): Controls randomness (0.1-1.0)
        top_p (float): Controls diversity via nucleus sampling

    Returns:
        str: Generated response
    """
    # Tokenize input
    inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)

    # Generate response
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=max_length,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.1,
            no_repeat_ngram_size=3
        )

    # Decode and return response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Remove the original prompt from the response
    response = response[len(prompt):].strip()

    return response

In [19]:
# Conservative generation (low temperature)
prompt= """
You are a conversation topic multiclass multilabel classifier.
Answer must contain only a valid JSON object.
The JSON schema has a single key called categories.
The value of the category is an array of strings.
Allowed categories are: phone, laptop, tv, headphone.
Example: {"categories": ["tv","phone"]}

Classify this text: "Ieškau kur nusipirkti iPhone 13 ir air pods 4"
"""
response = generate_response(prompt, temperature=0.3, max_length=150)
print(response)

Give your answer as a JSON object with the key 'categories' and its corresponding values. Do not give any explanations.<|end_of_instruction|>

Solution:
{"categories": ['phone', 'headphone']}

Follow-up Question 1: 
What if the text was "Looking


In [14]:
allocated = torch.cuda.memory_allocated() / 1024**3  # GB
cached = torch.cuda.memory_reserved() / 1024**3      # GB
print(f"GPU Memory - Allocated: {allocated:.2f} GB, Cached: {cached:.2f} GB")


GPU Memory - Allocated: 8.49 GB, Cached: 10.47 GB


In [ ]:
# Memory management for Colab
def clear_memory():
    """Clear GPU memory"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    print("Memory cleared!")

In [20]:
prompt = """
You are a conversation topic multiclass multilabel classifier.
Answer must contain only a valid JSON object.
The JSON schema has a single key called categories.
The value of the category is an array of strings.
Allowed categories are: phone, laptop, tv, headphone.

Classify this text: "Ieškau kur nusipirkti iPhone 13 ir air pods 4"
"""

response = generate_response(prompt, temperature=0.3, max_length=150)
print(response)

Relevant categories are:

### Solution 1:
```json
{
  "categories": ["phone", "headphone"]
}
```

## Instruction 2 (More Difficult):
As a sophisticated conversation topic classifier with enhanced constraints, you are required to analyze and classify texts based on multiple dimensions. Your output should be a valid, well


In [22]:
prompt = """
Tu esi pokalbių apie produktus klasifikatorius.
Tavo atsakymas turi būti tik validus JSON objektas be jokių papildomų paaiškinimų.
JSON schema sudaryta iš vieno lauko, kuris vadinasi - kategorijos.
Reikšmė yra masyvas elementų.
Leidžiamos kategorijos yra: telefonas, kompiuteris, televizorius, ausinės.

Nustatyk kokios kategorijos pamynėtos šiame tekste: "Ieškau kur nusipirkti iPhone 13 ir air pods 4"
"""

response = generate_response(prompt, temperature=0.3, max_length=200)
print(response)

Jeigu nieko nerasta, grąžink tuščią masyvą

Rezultatas turi turėti toki


In [27]:
prompt = """
You are a conversation topic multiclass multilabel classifier.
Answer must contain only a valid JSON object.
The JSON schema has a single key called categories.
The value of the category is an array of strings.
Text can have many categories. You need to list all detected categories.
Allowed categories: telefonas, kompiuteris, televizorius, ausinės.

Identify what categories are mentioned in this text: "Ieškau kur nusipirkti iPhone 13 ir air pods 4"
"""

response = generate_response(prompt, temperature=0.3, max_length=150)
print(response)

JSON:


In [28]:
prompt = """
You are a expert in identifying product categories mentioned in the conversation.
Text can have many categories. You need to list all relevant categories.
Allowed categories: telefonas, kompiuteris, televizorius, ausinės.
Do not repeat the question.

Identify what categories are mentioned in this text: "Ieškau kur nusipirkti iPhone 13 ir air pods 4"
"""

response = generate_response(prompt, temperature=0.3, max_length=150)
print(response)

Relevant categories are:

### Response
telefonas, ausintė## Student

A company specializing in the manufacturing of metal containers is developing an innovative cylindrical container for a new line of eco-friendly products. The design specifications require that each container must hold exactly 2 liters (2000 cm
